# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,3.48,59,95,2.11,US,1674943325
1,1,bambous virieux,-20.3428,57.7575,25.68,85,94,1.40,MU,1674943327
2,2,cherskiy,68.7500,161.3000,-29.82,97,100,1.85,RU,1674943328
3,3,new norfolk,-42.7826,147.0587,18.73,48,66,1.34,AU,1674943329
4,4,hobart,-42.8794,147.3294,18.92,64,75,3.60,AU,1674943330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.5,
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 20)
                            & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,ugoofaaru,5.6667,73.0000,26.33,77,0,3.41,MV,1674943367
60,60,castro,-24.7911,-50.0119,21.47,52,0,0.09,BR,1674943358
138,138,ongole,15.5000,80.0500,21.10,83,0,0.63,IN,1674943433
144,144,puerto escondido,15.8500,-97.0667,28.46,61,0,2.57,MX,1674943396
169,169,xai-xai,-25.0519,33.6442,23.27,91,0,0.57,MZ,1674943447
318,318,telhara,21.0269,76.8389,21.06,48,0,3.25,IN,1674943525
323,323,tecoanapa,16.5167,-98.7500,28.40,58,0,4.42,MX,1674943527
349,349,lima,-12.0432,-77.0282,22.14,78,0,3.60,PE,1674943500
469,469,paungde,18.4833,95.5000,20.12,61,0,1.97,MM,1674943602


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
48,ugoofaaru,MV,5.6667,73.0000,77,
60,castro,BR,-24.7911,-50.0119,52,
138,ongole,IN,15.5000,80.0500,83,
144,puerto escondido,MX,15.8500,-97.0667,61,
169,xai-xai,MZ,-25.0519,33.6442,91,
318,telhara,IN,21.0269,76.8389,48,
323,tecoanapa,MX,16.5167,-98.7500,58,
349,lima,PE,-12.0432,-77.0282,78,
469,paungde,MM,18.4833,95.5000,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # lat = row['Lat']
    # lon = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{row['Lng']},{row['Lat']},{radius}"
    bias = f"proximity:{row['Lng']},{row['Lat']}"
    params["filter"] = filters
    params["bias"] = bias

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index,
                     "Hotel"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel"] = "No hotel found"

    # Log the search results
    print(
        f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel']}")

# Display sample data
hotel_df


Starting hotel search
ugoofaaru - nearest hotel: No hotel found
castro - nearest hotel: Hotel Central Palace
ongole - nearest hotel: KOLAS Hotel
puerto escondido - nearest hotel: Hotel Barlovento
xai-xai - nearest hotel: No hotel found
telhara - nearest hotel: No hotel found
tecoanapa - nearest hotel: No hotel found
lima - nearest hotel: Hotel Peach Garden
paungde - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel
48,ugoofaaru,MV,5.6667,73.0000,77,No hotel found
60,castro,BR,-24.7911,-50.0119,52,Hotel Central Palace
138,ongole,IN,15.5000,80.0500,83,KOLAS Hotel
144,puerto escondido,MX,15.8500,-97.0667,61,Hotel Barlovento
169,xai-xai,MZ,-25.0519,33.6442,91,No hotel found
318,telhara,IN,21.0269,76.8389,48,No hotel found
323,tecoanapa,MX,16.5167,-98.7500,58,No hotel found
349,lima,PE,-12.0432,-77.0282,78,Hotel Peach Garden
469,paungde,MM,18.4833,95.5000,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.5,
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel", "Country", "Lat", "Lng"]
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel,Country)